Note that libraries that were used in the code is imported in "problem3.ipynb" where this file will be run.

### Functions for problem a

In [1]:
def boundaryfnc(x): #Boundary condition u(x,1) for the laplace equation
    return np.sin(2*np.pi*x)

def solutionfnc(x,y, mesh = True): #Analytic solution for problem 3a. Returns mesh
    if mesh:
        return np.outer(np.sin(2*np.pi*x),np.sinh(2*np.pi*y))/np.sinh(2*np.pi)
    return np.sin(2*np.pi*x)*np.sinh(2*np.pi*y)/np.sinh(2*np.pi)

#5-point formula taylored for problem 3a. Assumes g(0,x)=g(y,0)=g(y,1)=0. 
#boundfnc refers to the boundary condition u(x,1)=g(x)
def fivePointFormula(mx, my, boundfnc):
    
    #Step size and axis in x and y direction
    h=1/(mx+1)
    k=1/(my+1)

    #Define axies
    xaxis = np.linspace(0,1,mx+2)
    yaxis = np.linspace(0,1,my+2)
    
    #For shorter notation
    r = 2*(h**2+k**2)
    
    #Calculate diagonals for the system
    diag = -r*np.ones(mx*my)
    subsup = k**2*np.ones(mx*my-1)
    outer = h**2
    
    #Handle boundaries in the A matrix
    for i in range(mx,len(subsup),mx):
        subsup[i-1] = 0
    
    #Construct a sparse matrix for better computational time and storage
    A = diags([diag,subsup,subsup,outer,outer],[0,1,-1,mx,-mx])
    b = np.zeros(mx*my)
    
    #Calculate boundary values
    b[-mx:] = -h**2*boundfnc(xaxis[1:-1])
    
    #Solve the inner system with sparse
    solutionvec = sl.spsolve(A, b)
    
    #Make a new solution matrix since we need to reformat
    fixedSolution = np.zeros((mx+2,my+2))
    
    #Reconstruct the solution matrix
    for j in range(1,mx+1):
        for i in range(1, my+1):
            fixedSolution[j,i] = solutionvec[mx*(i-1)+j-1]
    
    #Impose the boundary conditions
    fixedSolution[-1,:] = 0
    fixedSolution[0,:] = 0
    fixedSolution[:,0] = 0
    fixedSolution[:,-1] = boundfnc(xaxis)
    
    #Return solution along with axies for plotting
    return fixedSolution, xaxis, yaxis

## Error plot function

In [2]:
#This function creates convergence plots.
#both = True yields plots where Mx=My
#x = True yields plots where My is fixed as endIndex and Mx varies
#x = False yields plots where Mx is fixed as endIndex and My varies
def errorPlots(startIndex = 1, endIndex = 2, x = True, both = False, evaluationPoints = 5, expectedOrder = 1, r = 1):
    
    if both: #My=Mx
        
        #Make a list of M-values we want to evaluate the method in
        Ms = np.logspace(startIndex, endIndex, evaluationPoints)
        
        errors = np.zeros(len(Ms)) #Stores the error estimates
        i=0
        for m in Ms:

            #Obtain the numerical solution
            numSol, xaxis, yaxis = fivePointFormula(int(m), int(m), boundaryfnc)

            #Obtain the corresponding analytic solution
            a = solutionfnc(xaxis, yaxis)
              
            #Save the error estimates
            errors[i] = np.linalg.norm(numSol-a)/np.linalg.norm(a)
            i+=1 #increment

        #Plotting
        Ms = Ms**2
        plt.loglog(Ms, errors, marker="." , label = "$\ell_2$-norm")
        plt.plot(np.logspace(np.log10(Ms[0]), np.log10(Ms[-1]), 5), np.logspace(np.log10(errors[0]), np.log10(errors[0])-expectedOrder*(np.log10(Ms[-1])-np.log10(Ms[0])), 5), "s", label="Expected order")
        plt.xlabel("Degrees of freedom")
        plt.ylabel("Relative $\ell_2$-error")
        plt.grid()
        plt.legend()
        plt.show()
        
    elif(x): #My is fixed, iterate over Mx

        #List of Mx values we want to evaluate for
        
        Mx = np.logspace(startIndex, endIndex, evaluationPoints)
        My = 2*Mx[-1] #Set My as the max value we want to evaluate so that the error in y does not interfere
        errors = np.zeros(len(Mx)) #Stores the error estimates
        i=0
              
        for mx in Mx: #Iterate
            
            #Obtain numerical solution
            numSol, xaxis, yaxis = fivePointFormula(int(My), int(mx), boundaryfnc)

            #Obtain analytic solution
            a = solutionfnc(xaxis,yaxis)
              
            #Store the error estimate
            errors[i] = np.linalg.norm(numSol-a)/np.linalg.norm(a)
            i+=1

        #Plotting
        plt.loglog(Mx, errors, marker="." ,label = "$\ell_2$-norm")
        plt.plot(np.logspace(np.log10(Mx[0]), np.log10(Mx[-1]), 5), np.logspace(np.log10(errors[0]), np.log10(errors[0])-expectedOrder*(np.log10(Mx[-1])-np.log10(Mx[0])), 5), "s", label="Expected order")
        plt.xlabel("Number of points along x-axis: $M$")
        plt.ylabel("Relative $\ell_2$-error")
        plt.grid()
        plt.legend()
        plt.show()

    #change discretization in y-direction
    elif(not x):
        
        #List of Mx values we want to evaluate for
        My = np.logspace(startIndex, endIndex, evaluationPoints)
        Mx = 2*My[-1] #Set the value of Mx to the max of My so that Mx does not interfare
        errors = np.zeros(len(My)) #Stores the error estimates
        i=0
        for my in My:
            
            #Obtain numerical solution
            numSol, xaxis, yaxis = fivePointFormula(int(my), int(Mx), boundaryfnc)

            #Obtain the analytic solution
            a = solutionfnc(xaxis,yaxis)
            
            #Store the error estimates
            errors[i] = np.linalg.norm(numSol-a)/np.linalg.norm(a)
            i+=1

        #Plotting
        plt.loglog(My, errors, marker="." , label = "$\ell_2$-norm")
        plt.plot(np.logspace(np.log10(My[0]), np.log10(My[-1]), 5), np.logspace(np.log10(errors[0]), np.log10(errors[0])-expectedOrder*(np.log10(My[-1])-np.log10(My[0])), 5), "s", label="Expected order")
        plt.xlabel("Number of points along y-axis: $M$")
        plt.ylabel("Relative $\ell_2$-error")
        plt.grid()
        plt.legend()
        plt.show() 